In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install keras_retinanet

## Imports

In [1]:
import os
import datetime
import errno
import argparse
import fnmatch

import numpy as np

from skimage.external.tifffile import imsave
from skimage.external.tifffile import TiffFile

import deepcell
from deepcell.utils.misc_utils import sorted_nicely
from deepcell.utils.tracking_utils import load_trks

## Track Multiple Movies with Multiple Parameters

### Define File Locations and Load the Benchmark Data to Test

In [2]:
# Define data to load (benchmark test folders)
bens_trks_3T3  = '/data/track_data/test_DC_SEG/tracked/3T3/'
bens_trks_HEK  = '/data/track_data/test_DC_SEG/tracked/HEK293/'
bens_trks_HeLa = '/data/track_data/test_DC_SEG/tracked/HeLa/'
bens_trks_RAW  = '/data/track_data/test_DC_SEG/tracked/RAW264.7/'

ben_trks_folders = [bens_trks_3T3, bens_trks_HEK, bens_trks_HeLa, bens_trks_RAW]

In [3]:
# Define destination folders for False Positive Corrected tracked batches
cor_tracked_bens_3T3  = '/data/track_data/Final_Benchmarks/DC_TEST_DC_SEG/3T3/'
cor_tracked_bens_HEK  = '/data/track_data/Final_Benchmarks/DC_TEST_DC_SEG/HEK293/'
cor_tracked_bens_HeLa = '/data/track_data/Final_Benchmarks/DC_TEST_DC_SEG/HeLa/'
cor_tracked_bens_RAW  = '/data/track_data/Final_Benchmarks/DC_TEST_DC_SEG/RAW264.7/'

cor_trk_ben_folders = [cor_tracked_bens_3T3, cor_tracked_bens_HEK, cor_tracked_bens_HeLa, cor_tracked_bens_RAW]

# Define a base file name for the output
BASE_NAME = 'batch_'

In [2]:
# ISBI data to load
bens_trks_HeLa  = '/data/track_data/ISBI_Challenge/Fluo-N2DL-HeLa/ForBenchmarking/'
#bens_trks_HeLa  = '/data/track_data/ISBI_Challenge_DC_SEG/tracked/HeLa/'

ben_trks_folders = [bens_trks_HeLa]

# ISBI destination folders for False Positive Corrected tracked batches
cor_tracked_bens_HeLa  = '/data/track_data/Final_Benchmarks/ISBI/Fluo-N2DL-HeLa_noFPP/'
#cor_tracked_bens_HeLa  = '/data/track_data/Final_Benchmarks/ISBI_DC_SEG/MaskRCNN/Fluo-N2DL-HeLa/'

cor_trk_ben_folders = [cor_tracked_bens_HeLa]

# Define a base file name for the output
BASE_NAME = 'batch_'

### Work Through Each Dataset with Multiple Parameters

#### Import a Tracking Model to Use

In [3]:
from deepcell import model_zoo

# Tracking model settings
features = {'appearance', 'distance', 'neighborhood', 'regionprop'}
min_track_length = 9
neighborhood_scale_size = 30
batch_size = 128  
crop_dim = 32

in_shape = (crop_dim, crop_dim, 1)

# Re-instantiate the tracking model
tracking_model = model_zoo.siamese_model(
    input_shape=in_shape,
    neighborhood_scale_size=neighborhood_scale_size,
    features=features)

# Load model weights
MODEL_DIR = '/data/models'
siamese_weights_file = 'tracking_model_benchmarking_757_step5_20epoch_80split_9tl.h5'
siamese_weights_file = os.path.join(MODEL_DIR, siamese_weights_file)

tracking_model.load_weights(siamese_weights_file)

#### Define Tracking Parameters

In [4]:
# Define Critical Parameters for Grid Search

# If model prediction for daughter is higher than this parameter, then daughter assignment made
#division=[0.5, 0.55, 0.6, 0.65, 0.7, 0.8, 0.9]
division=[0.9]

# Parameter for cost matrix - if other possible assignments are higher than this, then a birth occurred
#birth=[0.8, 0.85, 0.9, 0.95, 0.99]
birth=[0.99]

# Parameter for cost matrix - if other possible assignments are higher than this, then a death occurred
#death=[0.8, 0.85, 0.9, 0.95, 0.99]
death=[0.99]

# Define Parameters that will not be changed
max_distance=50

# Define Parameters that cannot be changed (because they are model dependent or training data dependent)
track_length=9
neighborhood_scale_size=30

#### Run the Model Iteratively for the Parameters of Interest

In [ ]:
# The tracking model is used in concert with other processes to track cells
# Import the neccesary tracking functionality
import deepcell.tracking

# The tracker requires normalized images as input
def image_norm(original_image):
    # NNs prefer input data that is 0 mean and unit variance
    normed_image = (original_image - np.mean(original_image)) / np.std(original_image)
    return normed_image

# Go through each Dataset (3T3, HEK293, HeLa, RAW264.7)
for set_num, dataset in enumerate(ben_trks_folders):
    # Go through each batch (movie) in each dataset
    movie_list = sorted_nicely(os.listdir(dataset))
    for batch_num, movie in enumerate(movie_list):
        # Load the trk file       
        filename = os.path.join(dataset, movie)
        trks = load_trks(filename)
        lineages, raw, tracked = trks["lineages"], trks["X"], trks["y"]
        
        # Normalize raw images
        for frame in range(raw.shape[0]):
            raw[frame, :, :, 0] = image_norm(raw[frame, :, :, 0]) 

        # Track each movie with each division parameter
        print('Tracking: ', filename)
        for div_param in division:
            # For each birth parameter
            for birth_param in birth:
                # For each death parameter
                for death_param in death:
                    # Track with the selected parameters
                    trial = deepcell.tracking.cell_tracker(raw, tracked,
                                         tracking_model, max_distance=max_distance, track_length=track_length, 
                                         division=div_param, birth=birth_param, death=death_param,
                                         neighborhood_scale_size=neighborhood_scale_size,
                                         features=features)
                    trial._track_cells()
                    # Run FP post processing and save the result in the correct location
                    file_name = BASE_NAME + str(batch_num).zfill(3)+'_div'+str(div_param).zfill(3)+'_b'+str(birth_param).zfill(3)+'_d'+str(death_param).zfill(3)+'.trk'
                    file_path = os.path.join(cor_trk_ben_folders[set_num], file_name)

                    #fp_fixed_trk = trial.postprocess(file_path)
                    trial.dump(file_path)



Tracking:  /data/track_data/ISBI_Challenge/Fluo-N2DL-HeLa/ForBenchmarking/batch_0.trk


/usr/local/lib/python3.5/dist-packages/skimage/measure/_regionprops.py:250: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See https://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)
/usr/local/lib/python3.5/dist-packages/skimage/measure/_regionprops.py:260: UserWarning: regionprops and image moments (including moments, normalized moments, central moments, and inertia tensor) of 2D images will change from xy coordinates to rc coordinates in version 0.16.
See https://scikit-image.org/docs/0.14.x/release_notes_and_installation.html#deprecations for details on how to avoid this message.
  warn(XY_TO_RC_DEPRECATION_MESSAGE)


Tracking frame 1
Tracked frame 1 in 6.6028862446546555 seconds.
Tracking frame 2
New track
New track
Division detected
Tracked frame 2 in 6.679745081812143 seconds.
Tracking frame 3
Tracked frame 3 in 6.817187283188105 seconds.
Tracking frame 4
New track
New track
Tracked frame 4 in 6.975821662694216 seconds.
Tracking frame 5
Tracked frame 5 in 7.0617163479328156 seconds.
Tracking frame 6
New track
Division detected
New track
New track
Division detected
New track
Tracked frame 6 in 7.362284105271101 seconds.
Tracking frame 7
Tracked frame 7 in 7.578709091991186 seconds.
Tracking frame 8
New track
Division detected
New track
Division detected
New track
New track
Tracked frame 8 in 7.630536511540413 seconds.
Tracking frame 9
Tracked frame 9 in 7.901914440095425 seconds.
Tracking frame 10
New track
Division detected
New track
Division detected
New track
New track
Tracked frame 10 in 8.240629374980927 seconds.
Tracking frame 11
New track
New track
Division detected
New track
New track
New 

## Translate the Results Into ISBI Format for Comparison

#### ISBI Translator Functions

In [ ]:
# Adds a new track to the lineage and swap the labels accordingly in the images
def create_new_ISBI_track(batch_tracked, batch_info, old_label, frames, daughters, frame_div):
    
    new_track = max(batch_info.keys())
    new_label = new_track + 1
         
    batch_info[new_label] = {}
    batch_info[new_label]['old_label'] = old_label
    batch_info[new_label]['label'] = new_label

    batch_info[new_label]['frames'] = frames
    batch_info[new_label]['daughters'] = daughters
    batch_info[new_label]['frame_div'] = frame_div
    batch_info[new_label]['parent'] = None

    for frame in frames:
        batch_tracked[frame][batch_tracked[frame] == old_label] = new_label
        
    return batch_info, batch_tracked

# Check for contiguous tracks (tracks should only consist of consecutive tracks)
# Split one track into two if neccesary
def contig_tracks(label, batch_info, batch_tracked):
    
    frame_div_missing = False
    
    original_label = label
    frames = batch_info[original_label]['frames']
    final_frame_idx = len(frames) - 1
       
    for frame_idx, frame in enumerate(frames):
        next_con_frame = frame + 1
        # If the next frame is available and contiguous we should move on to the next frame. Otherwise:
        # If the next frame is available and NONcontiguous we should separate this track into two 
        if frame_idx + 1 <= final_frame_idx and next_con_frame != frames[frame_idx + 1]:
            contig_end_idx = frame_idx

            next_trk_frames = frames[frame_idx+1:]
            daughters = batch_info[original_label]['daughters']
            
            
            if 'frame_div' in batch_info[original_label]:
                frame_div = batch_info[original_label]['frame_div']
            else:
                frame_div = None
                frame_div_missing = True
                                  
            # Create a new track to hold the information from this frame forward and add it to the batch
            batch_info, batch_tracked = create_new_ISBI_track(batch_tracked, batch_info, original_label, 
                                                                next_trk_frames, daughters, frame_div)
                        
            # Adjust the info for the current track to vacate the new track info
            batch_info[original_label]['frames'] = frames[0:contig_end_idx+1]
            batch_info[original_label]['daughters'] = []
            batch_info[original_label]['frame_div'] = None
                        
            # Because we are splitting tracks recursively, we stop here
            break
        
        # If the current frame is the last frame then were done
        # Either the last frame is contiguous and we don't alter batch_info
        # or it's not and it's been made into a new track by the previous iteration of the loop
        
    # Print warning if there is no 'frame_div'
    if frame_div_missing:
        print('Warning: frame_div is missing')
    
    return batch_info, batch_tracked

#### Define Save Location for Benchmark Files (ie: 001, 001_RES)

In [8]:
# Define where benchmark data will be saved
BENCH_DIR = '/data/tracking_benchmarks/Final_Benchmarks/DC_TEST_DC_SEG'

# Define where tracks will be saved
bench_3T3  = os.path.join(BENCH_DIR, '3T3/')
bench_HEK  = os.path.join(BENCH_DIR, 'HEK293/')
bench_HeLa = os.path.join(BENCH_DIR, 'HeLa/')
bench_RAW  = os.path.join(BENCH_DIR, 'RAW264.7/')

bench_tracked_folders = [bench_3T3, bench_HEK, bench_HeLa, bench_RAW]

In [ ]:
# For ISBI
# Define alt location where benchmark data will be saved
BENCH_DIR = '/data/tracking_benchmarks/Final_Benchmarks/ISBI/'
#BENCH_DIR = '/data/tracking_benchmarks/Final_Benchmarks/ISBI_DC_SEG/MaskRCNN/'

# Define where tracks will be saved
bench_HeLa = os.path.join(BENCH_DIR, 'Fluo-N2DL-HeLa_noFPP')
#bench_HeLa = os.path.join(BENCH_DIR, 'Fluo-N2DL-HeLa/')

bench_tracked_folders = [bench_HeLa]

#### Translate All trk files and Generate Benchmark Files

In [ ]:
## Create new benchmark files (ie: 001_RES) from mutliple .trk files
## NB: RAW and GT Files Should Already Exist


# Go through each Dataset (3T3, HEK293, HeLa, RAW264.7)
for set_num, dataset in enumerate(cor_trk_ben_folders):
    # Go through each batch (movie) in each dataset
    movie_list = sorted_nicely(os.listdir(dataset))
    
    for batch_num, batch in enumerate(movie_list):

        # Load the trk file
        filename = os.path.join(dataset, batch)
        trks = load_trks(filename)
        lineages, raw, tracked = trks["lineages"], trks["X"], trks["y"]

        # Define Save Location
        PARAMS = 'div'+str(div_param).zfill(3)+'_b'+str(birth_param).zfill(3)+'_d'+str(death_param).zfill(3)
        
        # Build subdirectories to hold benchmark info           
        BENCHMARK_DIR = os.path.abspath(os.path.join(bench_tracked_folders[set_num], PARAMS))
        B_SUB_DIR = os.path.join(BENCHMARK_DIR, '{:03}_RES'.format(batch_num+1))

        print('Files will be saved at: ', B_SUB_DIR)

        # create directories if they do not exist
        try:
            os.makedirs(B_SUB_DIR)
        except OSError as exc:  # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise

        # Record lineage data in txt as it is generated
        batch_info = lineages[0]

        # Prepare output txt
        text_file = open(os.path.join(B_SUB_DIR, "res_track.txt"), "w")

        batch_tracked = tracked
        labels = list(batch_info.keys())
        max_label = max(labels)

        labels_to_remove = []
        for label in labels:

            batch_info, batch_tracked = contig_tracks(label, batch_info, batch_tracked)

            first_frame = np.amin(batch_info[label]['frames'])          
            last_frame = np.amax(batch_info[label]['frames'])
            if batch_info[label]['parent']:
                parent = batch_info[label]['parent']
            else:
                parent = 0

            print(label, first_frame, last_frame, parent)
            text_file.write('{} {} {} {}\n'.format(label, first_frame, last_frame, parent))

            # Check if the track need to be split
            if max(batch_info.keys()) > max_label:
                # If so, a new track was added
                new_max_label = max(batch_info.keys())
                labels.append(new_max_label)
                max_label = new_max_label

        text_file.close()
        lineages[0] = batch_info
        tracked = batch_tracked


        # Save Image Files

        channel = 0 # These images should only have one channel
        for i in range(raw.shape[0]):
            #name_raw = os.path.join(B_SUB_DIR_RAW,'t{:03}_.tif'.format(i))
            name_tracked = os.path.join(B_SUB_DIR,'mask{:03}.tif'.format(i))
            #imsave(name_raw, raw[i, :, :, channel])
            imsave(name_tracked, tracked[i, :, :, channel].astype('uint16'))

#### Generate/Copy RAW and GT Benchmark Files to the Appropriate Locations

In [ ]:
import shutil, errno

def copy_raw_folders(src, dst):
    dir_name = os.path.basename(src)
    dst_path = os.path.join(dst, dir_name)
    
    # create directories if they do not exist
    try:
        os.makedirs(dst_path)
    except OSError as exc:  # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

    # copy each file over
    file_list = os.listdir(src)
    for src_file in file_list:
        src_path = os.path.join(src, src_file)
        shutil.copy(src_path, dst_path)
        
        
def copy_gt_folders(src, dst):
    
    dir_name = os.path.basename(src)
    dst_path_SEG = os.path.join(dst, dir_name, 'SEG')
    dst_path_TRA = os.path.join(dst, dir_name, 'TRA')
    
    # create directories if they do not exist
    for d in (dst_path_SEG, dst_path_TRA):
        try:
            os.makedirs(d)
        except OSError as exc:  # Guard against race condition
            if exc.errno != errno.EEXIST:
                raise


    # copy each file over
    file_list = os.listdir(os.path.join(src, 'SEG'))
    for src_file in file_list:
        src_path = os.path.join(src, 'SEG', src_file)
        shutil.copy(src_path, dst_path_SEG)
        
    file_list = os.listdir(os.path.join(src, 'TRA'))
    for src_file in file_list:
        src_path = os.path.join(src, 'TRA', src_file)
        shutil.copy(src_path, dst_path_TRA)



In [ ]:
## Create/Copy benchmark files (ie: 001, 001_GT, etc) from mutliple .trk files
import shutil

# Define all the original RAW/GT Benchmark Folders
# bens_folders_3T3  = '/data/tracking_benchmarks/Viterbi_test/GT_Tracking/3T3/NIH/'
# bens_folders_HEK  = '/data/tracking_benchmarks/Viterbi_test/GT_Tracking/HEK293/generic/'
# bens_folders_HeLa = '/data/tracking_benchmarks/Viterbi_test/GT_Tracking/HeLa/S3/'
# bens_folders_RAW  = '/data/tracking_benchmarks/Viterbi_test/GT_Tracking/RAW264.7/generic/'

# bens_folders_list = [bens_folders_3T3, bens_folders_HEK, bens_folders_HeLa, bens_folders_RAW]

# ISBI
# Define all the original RAW/GT Benchmark Folders
bens_folders_HeLa = '/data/tracking_benchmarks/Viterbi_ISBI/GT_Tracking/Fluo-N2DL-HeLa'

bens_folders_list = [bens_folders_HeLa]


# Go through each Dataset (3T3, HEK293, HeLa, RAW264.7)
for ben_idx, ben_folder in enumerate(bens_folders_list):
    folder_list = os.listdir(ben_folder)
    folder_list_sorted = sorted_nicely(folder_list)
    
    # Build two lists - 1 that holds all the RAW batches and 1 that holds all GT batches
    # These will be the source files to copy
    RAW_folders = []
    GT_folders = []
    # Loop through all of the folders in the directory
    for folder in folder_list_sorted:
        # First select folders containing raw images
        if fnmatch.fnmatch(folder, '???'):
            raw_folder = os.path.join(ben_folder,folder)
            RAW_folders.append(raw_folder)
        # Next select folders containing GT images
        elif fnmatch.fnmatch(folder, '???_GT'):
            GT_folder = os.path.join(ben_folder,folder)
            GT_folders.append(GT_folder)
            
    # Copy the raw and GT files for each batch (movie) in each dataset
    for src_raw, src_gt in zip(RAW_folders, GT_folders):
                
        # Define the destination folders (1 for each paramter combination)
        # Loop through each of these folders and copy
        for div_param in division:
            # For each birth parameter
            for birth_param in birth:
                # For each death parameter
                for death_param in death:
                                        
                    PARAMS = 'div'+str(div_param)+'_b'+str(birth_param)+'_d'+str(death_param)
                    BENCHMARK_DIR = os.path.join(bench_tracked_folders[ben_idx], PARAMS)
                    dest = BENCHMARK_DIR
 
                    copy_raw_folders(src_raw, dest)
                    copy_gt_folders(src_gt, dest)

## Run Graph Benchmarks

#### Graph Comparison Code

In [12]:
import operator
from collections import Counter
import networkx as nx
import pandas as pd
import numpy as np
import glob

from skimage.measure import regionprops
from skimage.external.tifffile import TiffFile
from deepcell.utils.compute_overlap import compute_overlap
 
    

def create_graph(file, node_key=None):
    df = pd.read_csv(file, header=None, sep=' ', names=['Cell_ID','Start','End','Parent_ID'])
    
    if node_key != None:
        df[['Cell_ID','Parent_ID']] = df[['Cell_ID','Parent_ID']].replace(node_key)    
    
    edges = pd.DataFrame()    
    
    # Add each cell lineage as a set of edges to df    
    for _, row in df.iterrows():
        tpoints = np.arange(row['Start'], row['End']+1)
        deltaT = len(tpoints)        
        
        cellid = ['{cellid}_{frame}'.format(cellid = row['Cell_ID'], frame = t) for t in tpoints]
        source = cellid[0:-1]
        target = cellid[1:]        
        
        edges = edges.append(pd.DataFrame({'source':source, 'target':target}))    
        
    Dattr = {}
    # Add parent-daughter connections
    for _,row in df[df['Parent_ID']!=0].iterrows():
        source = '{cellid}_{frame}'.format(cellid = row['Parent_ID'], frame = row['Start']-1)
        target = '{cellid}_{frame}'.format(cellid = row['Cell_ID'], frame = row['Start'])        
        
        edges = edges.append(pd.DataFrame({'source':[source], 'target':[target]}))
        
        Dattr[source] = {'division':True}    
        
    # Create graph
    G = nx.from_pandas_edgelist(edges, source='source', target='target')
    nx.set_node_attributes(G, Dattr)
    return G

def load_data(pattern):
    files = np.sort(glob.glob(pattern))
    Lim = []
    for i,f in enumerate(files):
        Lim.append(TiffFile(f).asarray())    
        
    im = np.stack(Lim)
    return(im)



def match_nodes(pattern1, pattern2):
    gt = load_data(pattern1)
    res = load_data(pattern2)
    
    num_frames = gt.shape[0]
    iou = np.zeros((num_frames, np.max(gt)+1, np.max(res)+1))   
     
    # Compute IOUs only when neccesary
    # If bboxs for true and pred do not overlap with each other, the assignment is immediate
    # Otherwise use pixel-wise IOU to determine which cell is which
    
    # Regionprops expects one frame at a time
    for frame in range(num_frames):
        gt_frame = gt[frame]
        res_frame = res[frame]
        
        gt_props = regionprops(np.squeeze(gt_frame.astype('int')))
        gt_boxes = [np.array(gt_prop.bbox) for gt_prop in gt_props]
        gt_boxes = np.array(gt_boxes).astype('double')
        gt_box_labels = [int(gt_prop.label) for gt_prop in gt_props]
        
        res_props = regionprops(np.squeeze(res_frame.astype('int')))
        res_boxes = [np.array(res_prop.bbox) for res_prop in res_props]
        res_boxes = np.array(res_boxes).astype('double')
        res_box_labels = [int(res_prop.label) for res_prop in res_props]
        
        overlaps = compute_overlap(gt_boxes, res_boxes)    # has the form [gt_bbox, res_bbox]
        
        # Find the bboxes that have overlap at all (ind_ corresponds to box number - starting at 0)
        ind_gt, ind_res = np.nonzero(overlaps)
               
        #frame_ious = np.zeros(overlaps.shape)
        for index in range(ind_gt.shape[0]):
                        
            iou_gt_idx = gt_box_labels[ind_gt[index]]
            iou_res_idx = res_box_labels[ind_res[index]]
            intersection = np.logical_and(gt_frame==iou_gt_idx, res_frame==iou_res_idx)
            union = np.logical_or(gt_frame==iou_gt_idx, res_frame==iou_res_idx)            
            iou[frame, iou_gt_idx, iou_res_idx] = intersection.sum() / union.sum()
                
    gtcells, rescells = np.where(np.nansum(iou,axis=0)>=1)
    
    
    return gtcells, rescells

def classify_divisions(G_gt,G_res):    
    
    # Identify nodes with parent attribute
    div_gt =[node for node,d in G_gt.node.data() if d.get('division',False) == True]
    div_res =[node for node,d in G_res.node.data() if d.get('division',False) == True]    
    
    divI = 0 # Correct division
    divJ = 0 # Wrong division
    divC = 0 # False positive division
    divGH = 0 # Missed division    
    
    for node in div_gt:
        nb_gt = list(G_gt.neighbors(node))        
        
        # Check if res node was also called a division
        if node in div_res:
            nb_res = list(G_gt.neighbors(node))
            # If neighbors are same, then correct division
            if Counter(nb_gt) == Counter(nb_res):
                divI += 1
            # Wrong division
            elif len(nb_res) == 3:
                divJ += 1
            else:
                divGH += 1
        # If not called division, then missed division
        else:
            divGH += 1        
            
        # Remove processed nodes from res list
        try:
            div_res.remove(node)
        except:
            print('attempted removal of node {} failed'.format(node))    
            
    # Count any remaining res nodes as false positives   
    divC += len(div_res)   
    
    return({'Correct division': divI,
            'Incorrect division': divJ,
            'False positive division':divC,
            'False negative division':divGH})

#### Iteratively Compare Each Movie (Tracked with Each Parameter) To the Ground Truth Data

In [31]:
#############################################################################
#####
#####        LOAD/CHANGE BENCHMARK DIRECTORIES IF NECESSARY
#####
#############################################################################

# Define where benchmark data will be saved
#BENCH_DIR = '/data/tracking_benchmarks/DC_Seg_Test'
#BENCH_DIR = '/data/tracking_benchmarks/DC_test_final_param_FP_cor/'
#BENCH_DIR = '/data/tracking_benchmarks/DC_test_param_sol_FP_post/'
BENCH_DIR = '/data/tracking_benchmarks/MORDOR_TEST/OldAcc/'
#BENCH_DIR = '/data/tracking_benchmarks/Viterbi_test/KTH_Seg_Track'

# Define where tracks will be saved
# bench_3T3  = os.path.join(BENCH_DIR, '3T3/NIH')
# bench_HEK  = os.path.join(BENCH_DIR, 'HEK293/generic')
# bench_HeLa = os.path.join(BENCH_DIR, 'HeLa/S3')
# bench_RAW  = os.path.join(BENCH_DIR, 'RAW264.7/generic')

bench_3T3  = os.path.join(BENCH_DIR, '3T3/')
bench_HEK  = os.path.join(BENCH_DIR, 'HEK293/')
bench_HeLa = os.path.join(BENCH_DIR, 'HeLa/')
bench_RAW  = os.path.join(BENCH_DIR, 'RAW264.7/')

bench_tracked_folders = [bench_3T3, bench_HEK, bench_HeLa, bench_RAW]

#BENCH_DIR = '/data/tracking_benchmarks/Viterbi_ISBI/DL_Tracking'

# Define where tracks will be saved
#bench_MSC  = os.path.join(BENCH_DIR, 'Fluo-C2DL-MSC')
#bench_HeLa  = os.path.join(BENCH_DIR, 'Fluo-N2DL-HeLa')
#bench_U373 = os.path.join(BENCH_DIR, 'PhC-C2DH-U373')

#bench_tracked_folders = [bench_MSC, bench_HeLa, bench_U373]
#bench_tracked_folders = [bench_HeLa]


In [13]:
#############################################################################
#####
#####       FOR MULTIPLE PARAMETERS:
#####

# Prep a list to hold each dataset's results
dataset_cm = []

# Go through each Dataset (3T3, HEK293, HeLa, RAW264.7)
for ben_trk_folder in bench_tracked_folders:

    # Prep a list to hold results for each set of parameters
    params_cm = []
   
    # Go through each parameter combination
    for div_param in division:
        # For each birth parameter
        for birth_param in birth:
            # For each death parameter
            for death_param in death:  

                PARAMS = 'div'+str(div_param)+'_b'+str(birth_param)+'_d'+str(death_param)
                BENCHMARK_DIR = os.path.join(ben_trk_folder, PARAMS)
                
                # Compile a list of each movie
                sub_dirs = sorted_nicely(os.listdir(BENCHMARK_DIR))
                movie_list = fnmatch.filter(sub_dirs, '???')
                # Prep a list to hold each movie's results
                cm_list = []
                # Loop through each set of movies 
                for name in movie_list:
                    # Extract track.txt for each movie
                    pattern_gt = os.path.join(BENCHMARK_DIR, name + '_GT/TRA/')
                    pattern_res = os.path.join(BENCHMARK_DIR, name + '_RES/')

                    gtcells, rescells = match_nodes(pattern_gt+'*.tif',pattern_res+'*.tif')

                    if len(np.unique(rescells)) < len(np.unique(gtcells)):
                        node_key = {r:g for g,r in zip(gtcells,rescells)}
                        # node_key
                        # Maps gt nodes onto resnodes so must be applied to gt
                        G_res = create_graph(pattern_res+'res_track.txt',node_key=node_key)
                        G_gt = create_graph(pattern_gt+'man_track.txt')
                        cm_list.append(classify_divisions(G_gt,G_res))
                    else:
                        node_key = {g:r for g,r in zip(gtcells,rescells)}
                        G_res = create_graph(pattern_res+'res_track.txt')
                        G_gt = create_graph(pattern_gt+'man_track.txt',node_key=node_key)
                        cm_list.append(classify_divisions(G_gt,G_res))            

                params_cm.append(cm_list)
                
    dataset_cm.append(params_cm)

attempted removal of node 31_20 failed
attempted removal of node 173_73 failed
attempted removal of node 322_62 failed
attempted removal of node 124_55 failed
attempted removal of node 6_29 failed
attempted removal of node 29_13 failed
attempted removal of node 50_20 failed
attempted removal of node 132_60 failed
attempted removal of node 118_49 failed
attempted removal of node 22_51 failed
attempted removal of node 15_10 failed
attempted removal of node 119_67 failed
attempted removal of node 206_86 failed
attempted removal of node 11_42 failed
attempted removal of node 20_36 failed
attempted removal of node 474_59 failed
attempted removal of node 8_75 failed
attempted removal of node 66_34 failed
attempted removal of node 43_9 failed
attempted removal of node 495_63 failed
attempted removal of node 185_78 failed
attempted removal of node 48_29 failed
attempted removal of node 150_33 failed
attempted removal of node 26_9 failed
attempted removal of node 53_24 failed
attempted removal 

In [27]:
#############################################################################
#####
#####       FOR SINGlE PARAMETERS:
#####

# Prep a list to hold each dataset's results
dataset_cm = []

# Go through each Dataset (3T3, HEK293, HeLa, RAW264.7)
for ben_trk_folder in bench_tracked_folders:

    # Prep a list to hold results for each set of parameters
    params_cm = []
   
    BENCHMARK_DIR = os.path.join(ben_trk_folder)

    # Compile a list of each movie
    sub_dirs = sorted_nicely(os.listdir(BENCHMARK_DIR))
    movie_list = fnmatch.filter(sub_dirs, '???')
    # Prep a list to hold each movie's results
    cm_list = []
    # Loop through each set of movies 
    for name in movie_list:
        # Extract track.txt for each movie
        pattern_gt = os.path.join(BENCHMARK_DIR, name + '_GT/TRA/')
        pattern_res = os.path.join(BENCHMARK_DIR, name + '_RES/')

        gtcells, rescells = match_nodes(pattern_gt+'*.tif',pattern_res+'*.tif')

        if len(np.unique(rescells)) < len(np.unique(gtcells)):
            node_key = {r:g for g,r in zip(gtcells,rescells)}
            # node_key
            # Maps gt nodes onto resnodes so must be applied to gt

            G_res = create_graph(pattern_res+'res_track.txt',node_key=node_key)
            G_gt = create_graph(pattern_gt+'man_track.txt')
            cm_list.append(classify_divisions(G_gt,G_res))
        else:
            node_key = {g:r for g,r in zip(gtcells,rescells)}
            G_res = create_graph(pattern_res+'res_track.txt')
            G_gt = create_graph(pattern_gt+'man_track.txt',node_key=node_key)
            cm_list.append(classify_divisions(G_gt,G_res))            

    params_cm.append(cm_list)
                
    dataset_cm.append(params_cm)

attempted removal of node 11_5 failed
attempted removal of node 17_13 failed
attempted removal of node 3_3 failed
attempted removal of node 15_23 failed
attempted removal of node 2_34 failed
attempted removal of node 8_7 failed
attempted removal of node 15_17 failed


In [14]:
cm_totals = []
for cell_type in dataset_cm:
    cm_params = []
    for param_comb in cell_type:

        divI,divJ,divC,divGH = 0, 0, 0, 0
        dataset_stats = {'Correct division': divI,
                         'Incorrect division': divJ,
                         'False positive division':divC,
                         'False negative division':divGH}
        for cm in param_comb:
            dataset_stats['Correct division'] = dataset_stats['Correct division']+cm['Correct division']
            dataset_stats['Incorrect division'] = dataset_stats['Incorrect division']+cm['Incorrect division']
            dataset_stats['False positive division'] = dataset_stats['False positive division']+cm['False positive division']
            dataset_stats['False negative division'] = dataset_stats['False negative division']+cm['False negative division']

        cm_params.append(dataset_stats)
    
    cm_totals.append(cm_params)
    #         print(cm['False negative division'])
    #         print(cm['False positive division'])
    #         print(cm['Incorrect division'])

In [15]:
total_ideal_params = []
for cell_idx, cell_type in enumerate(cm_totals):
    min_FP = 1000
    min_FP_idx = []
    min_ID = 1000
    min_ID_idx = []
    min_FN = 1000
    min_FN_idx = []
    max_CD = 0
    max_CD_idx = []
    for param_idx, param_comb in enumerate(cell_type):
        if param_comb['Correct division'] >= max_CD:
            max_CD = param_comb['Correct division']
            max_CD_idx.append(param_idx)
        
        if param_comb['Incorrect division'] <= min_ID:
            min_ID = param_comb['Incorrect division']
            min_ID_idx.append(param_idx)
        
        if param_comb['False positive division'] <= min_FP:
            min_FP = param_comb['False positive division']
            min_FP_idx.append(param_idx)
        
        if param_comb['False negative division'] <= min_FN:
            min_FN = param_comb['False negative division']
            min_FN_idx.append(param_idx)

    print('Dataset ',cell_idx)
    print('MAX CD')
    print(max_CD, ' at ', max_CD_idx)
    print('MIN ID')
    print(min_ID, ' at ', min_ID_idx)
    print('min_FP')
    print(min_FP, ' at ', min_FP_idx)
    print('min_FN')
    print(min_FN, ' at ', min_FN_idx)
    
    ideal_params = []
    for param_idx, param_comb in enumerate(cell_type):
        if param_comb['Correct division'] == max_CD and param_comb['Incorrect division'] == min_ID and param_comb['False positive division'] == min_FP and param_comb['False negative division'] == min_FN:
            ideal_params.append(param_idx)
            
    total_ideal_params.append(ideal_params)

Dataset  0
MAX CD
204  at  [0]
MIN ID
0  at  [0]
min_FP
9  at  [0]
min_FN
102  at  [0]


In [16]:
counter = 0
# Go through each parameter combination
for div_param in division:
    # For each birth parameter
    for birth_param in birth:
        # For each death parameter
        for death_param in death:  

            if counter == 0:
                PARAMS = 'div'+str(div_param)+'_b'+str(birth_param)+'_d'+str(death_param)
                print(PARAMS)
                print('3T3')
                print(cm_totals[0][counter])
                print('HEK293')
                print(cm_totals[1][counter])
                print('HeLa')
                print(cm_totals[2][counter])
                print('RAW')
                print(cm_totals[3][counter])


                
            counter = counter + 1

div0.9_b0.99_d0.99
3T3
{'False negative division': 102, 'Incorrect division': 0, 'Correct division': 204, 'False positive division': 9}
HEK293


IndexError: list index out of range

## Run ISBI Benchmarking Scripts 

In [17]:
import subprocess
import statistics

# Define path to folder containing the directories of interest (ie: 001, 001_GT, 001_RES)

# ISBI_bens_3T3  = '/data/tracking_benchmarks/Final_Benchmarks/DC_TEST_DC_SEG/3T3/div0.9_b0.99_d0.99/'
# ISBI_bens_HEK  = '/data/tracking_benchmarks/Final_Benchmarks/DC_TEST_DC_SEG/HEK293/div0.9_b0.99_d0.99/'
# ISBI_bens_HeLa = '/data/tracking_benchmarks/Final_Benchmarks/DC_TEST_DC_SEG/HeLa/div0.9_b0.99_d0.99/'
# ISBI_bens_RAW  = '/data/tracking_benchmarks/Final_Benchmarks/DC_TEST_DC_SEG/RAW264.7/div0.9_b0.99_d0.99/'

# ISBI_bens_folders = [ISBI_bens_3T3, ISBI_bens_HEK, ISBI_bens_HeLa, ISBI_bens_RAW]
# ISBI_bens_folder_names = ['3T3', 'HEK293', 'HeLa', 'RAW']


# Alt path for ISBI dataset
ISBI_bens_HeLa  = '/data/tracking_benchmarks/Final_Benchmarks/ISBI_DC_SEG/MaskRCNN/Fluo-N2DL-HeLa/div0.9_b0.99_d0.99/'

ISBI_bens_folders = [ISBI_bens_HeLa]
ISBI_bens_folder_names = ['HeLa']



for index, path in enumerate(ISBI_bens_folders):

    # Calculate the number of batches
    dirs = os.listdir(path)
    num_batches = int(len(dirs)/3)

    TRA_Vals = []
    for batch in range(num_batches):
        batch = '{:03}'.format(batch+1)

        # Run ISBI Tracking Benchmark
        p = subprocess.run(['./TRAMeasure', path, batch], stdout=subprocess.PIPE)
        # Save the output
        outstring = p.stdout

        try:
            TRA_Val = float(outstring.decode('utf-8').split()[-1])
            TRA_Vals.append(TRA_Val)
        except:
            print('Benchmarking failure - Batch ', batch)
            print(outstring.decode('utf-8'))
    
    print('Dataset: ', ISBI_bens_folder_names[index])
    print('Mean: ', statistics.mean(TRA_Vals))
    print('Std. Dev.: ', statistics.pstdev(TRA_Vals))

Dataset:  HeLa
Mean:  0.904137
Std. Dev.:  0.03951699999999997
